In [2]:
import asyncio
import time
import os
from dataclasses import dataclass
from typing import Literal, Union
import httpx
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel




from dotenv import load_dotenv
load_dotenv('/home/dsa/quizbee/envs/.env', override=True)

# Verify keys are loaded
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
GROK_API_KEY = os.environ.get("GROK_API_KEY")


### Streaming + structured output - DOESNT WORK!



In [10]:
class QuizItemAnswer(BaseModel):
    answer: str = Field(description="The answer text")
    explanation: str = Field(description="The explanation text")
    correct: bool = Field(description="Whether the answer is correct")


class QuizItemTest(BaseModel):
    question: str = Field(description="The quiz question text")
    answers: list[QuizItemAnswer] = Field(
        description="4 answers with only one correct",
        min_length=4,
        max_length=4
    )


class TestOutput(BaseModel):
    mode: Literal["quiz"] = "quiz"
    quiz_items: list[QuizItemTest] = Field(
        description="Array of quiz items",
        min_length=5,
        max_length=5
    )

    
@dataclass
class TestDeps:
    """Dependencies for the test agent"""
    query: str


In [16]:
# Simple streaming example - just one model, one run
print("🚀 Starting streaming demo with Grok...\n")
print("="*60)

agent = Agent(
    model="grok:grok-4-fast-non-reasoning",
    deps_type=TestDeps,
)

start_time = time.time()
first_token_time = None

async with agent.run_stream(
    "Generate 5 quiz questions about: Python basics",
    deps=TestDeps(query="Python basics"),
    output_type=TestOutput
) as result:
    print("\n📝 Response (streaming):\n")
    
    # For structured output, we need to stream the underlying messages
    async for message in result.stream_output():
        if first_token_time is None:
            first_token_time = time.time() - start_time
            print(f"⚡ First token arrived in {first_token_time:.2f}s\n")
        
        # Print the delta content if it's a text message
        if hasattr(message, 'content') and isinstance(message.content, str):
            print(message.content, end="", flush=True)
        elif hasattr(message, 'parts'):
            for part in message.parts:
                if hasattr(part, 'content') and isinstance(part.content, str):
                    print(part.content, end="", flush=True)
    
    print("\n")
    
    # Get structured output - simply await it
    output = await result.get_output()
    
total_time = time.time() - start_time

print("="*60)
print(f"✅ Total time: {total_time:.2f}s")
print(f"⏱️  Time to first token: {first_token_time:.2f}s" if first_token_time else "⏱️  No streaming chunks received")
print(f"📊 Generated {len(output.quiz_items)} quiz questions")

# Show the structured output
print("\n📋 Structured Output:")
for i, item in enumerate(output.quiz_items, 1):
    print(f"\nQuestion {i}: {item.question}")
    for j, answer in enumerate(item.answers, 1):
        check = "✓" if answer.correct else "✗"
        print(f"  {check} {j}. {answer.answer}")


🚀 Starting streaming demo with Grok...


📝 Response (streaming):

⚡ First token arrived in 2.83s



✅ Total time: 2.83s
⏱️  Time to first token: 2.83s
📊 Generated 5 quiz questions

📋 Structured Output:

Question 1: What is the correct way to create a variable in Python?
  ✓ 1. x = 5
  ✗ 2. let x = 5
  ✗ 3. var x = 5
  ✗ 4. int x = 5

Question 2: Which of the following is a correct Python comment?
  ✓ 1. # This is a comment
  ✗ 2. // This is a comment
  ✗ 3. /* This is a comment */
  ✗ 4. -- This is a comment

Question 3: How do you print 'Hello, World!' in Python?
  ✓ 1. print('Hello, World!')
  ✗ 2. echo('Hello, World!')
  ✗ 3. console.log('Hello, World!')
  ✗ 4. printf('Hello, World!')

Question 4: What is the output of print(2 + 3 * 4) in Python?
  ✗ 1. 20
  ✓ 2. 14
  ✗ 3. 5
  ✗ 4. 24

Question 5: Which data type is used to store a sequence of characters in Python?
  ✓ 1. string
  ✗ 2. list
  ✗ 3. integer
  ✗ 4. boolean

📝 Response (streaming):

⚡ First token arrived in 2.83s



✅ T